In [2]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

/Users/sem.kolesnikov/proj/TextsGraphCompare/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 127kB/s]
README.md: 100%|██████████| 8.66k/8.66k [00:00<00:00, 7.54MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 106kB/s]
data_config.json: 100%|██████████| 25.5k/25.5k [00:00<00:00, 6.41MB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [01:12<00:00, 6.02MB/s] 
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 21.9kB/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<00:00, 78.8kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.20MB/s]
tokenizer_config.json: 100%|██████████| 363/363 [00:00<00:00, 146kB/s]
train_script.py: 100%|██████████| 13.9k/13.9k [00:00<00:00, 5.17MB

The cat sits outside 		 The dog plays in the garden 		 Score: 0.6146
A man is playing guitar 		 A woman watches TV 		 Score: 0.5309
The new movie is awesome 		 The new movie is so great 		 Score: 0.9582


In [128]:
from gensim.models import KeyedVectors

In [130]:
model = KeyedVectors.load_word2vec_format('cc.ru.300.vec.gz', binary=False)

In [30]:
import json


with open('data/docs.json') as f:
    raw_docs = json.load(f)

In [31]:
from data_reader import Document


raw_docs = [Document(key, value.split()) for key, value in raw_docs.items()]

In [10]:
import tqdm


doc_to_embed = {}
for doc_name, text in tqdm.tqdm(raw_docs.items()):
    doc_to_embed[doc_name] = model.encode(text)

100%|██████████| 1236/1236 [15:59<00:00,  1.29it/s]


In [17]:
import common
from importlib import reload
common = reload(common)

In [21]:
import networkx as nx
import scoring
from functools import partial
from common import cosine_sim, get_graph

1236it [00:16, 74.33it/s] 


In [133]:
graph = get_graph(raw_docs, 0.8, partial(cosine_sim, doc_to_vec=doc_to_embed))

doc_name_to_doc = {doc.name: doc for doc in raw_docs}

components = list(nx.connected_components(graph))
len(components)
components_docs = [
    Document(k, sum((doc_name_to_doc[doc_name].tokens for doc_name in docs_names), []))
    for k, docs_names in enumerate(components)
]

1236it [00:12, 96.89it/s] 


In [94]:
component_to_topics = scoring.get_topics_ctfidf(components_docs, reduce_frequent_words=True, bm25_weighting=True, top_k=10, min_df=0.3, max_df=0.8)
components_topics = [v for k, v in component_to_topics.items()]

39it [00:00, 104.14it/s]


In [132]:
common.compute_top_words_sim(components_topics, model, topn=10)

0.20811919490389272